In [ ]:
import torchvision
import torch
import sys

sys.path.append("..")

from torch.utils.data import DataLoader, Dataset
from utils.model_util import LeNet5, CNN, MLP
from utils.train_util import *

model = LeNet5(28, 28, 1, 10).cuda()
train_dataset_loader = DataLoader(
    dataset=torchvision.datasets.MNIST(
        root='../data/raw-data/',
        train=True,
        transform=torchvision.transforms.ToTensor()),
    batch_size=600,
    shuffle=False)
test_dataset_loader = DataLoader(
    dataset=torchvision.datasets.mnist.MNIST(
        root='../data/raw-data/',
        transform=torchvision.transforms.ToTensor()),
    batch_size=32,
    shuffle=False)
test_dataset = torchvision.datasets.mnist.MNIST(
    root='../data/raw-data/',
    transform=torchvision.transforms.ToTensor())

opti = torch.optim.Adam(model.parameters())
criterion = torch.nn.CrossEntropyLoss(reduction='sum')
for data, target in train_dataset_loader:
    opti.zero_grad()
    output = model(data.cuda())
    loss = criterion(output, target.cuda())
    loss.backward()
    print(loss.item())
    opti.step()

print(loss, type(loss))
print(loss.item(), type(loss.item()))

model = LeNet5(28, 28, 1, 10).cuda()
train_dataset_loader = DataLoader(
    dataset=torchvision.datasets.MNIST(
        root='../data/raw-data/',
        train=True,
        transform=torchvision.transforms.ToTensor()),
    batch_size=600,
    shuffle=False)
test_dataset_loader = DataLoader(
    dataset=torchvision.datasets.mnist.MNIST(
        root='../data/raw-data/',
        transform=torchvision.transforms.ToTensor()),
    batch_size=32,
    shuffle=False)

opti = torch.optim.Adam(model.parameters())
criterion = torch.nn.CrossEntropyLoss(reduction='sum')
loss_sum = 0
for data, target in train_dataset_loader:
    opti.zero_grad()
    output = model(data.cuda())
    loss = criterion(output, target.cuda())
    # print(f'loss_sum = {loss_sum:<20}', end=' | ')
    loss.backward()
    loss_sum += loss.item()
    # print(f'loss = {loss.item():<20}')
    opti.step()

def eval_model(model, dataset, device):
    '''
    评估模型
    '''
    model_copy = deepcopy(model).eval()
    correct = 0
    data_loader = DataLoader(dataset, batch_size=32)
    for images, targets in data_loader:
        outputs = model_copy(images.to(device))
        _, predicted = torch.max(outputs, 1)
        correct += torch.eq(predicted, targets.to(device)).sum()
    # print(f'Test Accuracy: {100 * correct / total:.2f}%')
    accuracy = correct / len(dataset)
    return accuracy

eval_model(model, test_dataset, 'cuda')

model_copy = deepcopy(model).eval()
model_copy.eval()
model_copy.cuda()
correct = 0
total = 0
data_loader = test_dataset_loader
for images, targets in data_loader:
    outputs = model_copy(images.cuda())
    _, predicted = torch.max(outputs.data, 1)
    total += targets.size(0)
    correct += (predicted == targets.cuda()).sum().item()
    break
print('Test Accuracy: {:.2f}%'.format(100 * correct / total))

In [ ]:
for i in range(8, 13):
    print(f'{i:<10}|')
    print(f'{i:.2f}|')

In [ ]:
def get_list(len_list, term=[]):
    '''
    返回一个全是空列表的列表
    '''
    list_return = []
    for _ in range(len_list):
        list_return.append(term)
    return list_return

num_all_client = 6
num_all_server = 2

server_model = get_list(num_all_server)
server_accuracy = get_list(num_all_server)
client_accuracy = get_list(num_all_client)
server_client_model = get_list(num_all_server)
server_model_distillation_accuracy = get_list(num_all_server)

print(client_accuracy)

In [4]:
import torch
import sys

sys.path.append('..')

from utils.data_util import *
from utils.train_util import *
from utils.model_util import *
from utils.lib_util import *

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
print(device)

save_path = '../data/dealed-data/'
train_dataset_o, test_dataset_o = get_dataset(dataset='cifar10')
train_dataset_splited = split_data(train_dataset_o)
idx_client_target = train_data_split(train_dataset_splited, range(3))
all_target = train_dataset_splited.keys()

train_dataset_client = list_same_term(3)
for client in range(3):
    for target in all_target:
        train_dataset_client_new = [train_dataset_splited[target][idx]
                                    for idx in idx_client_target[client][target]]
        train_dataset_client[client].extend(train_dataset_client_new)

public_idx, test_idx = split_idx_proportion(
    number_list(len(test_dataset_o)), [0.2, 0.8])
public_dataset = DealDataset(test_dataset_o, public_idx)
test_dataset = DealDataset(test_dataset_o, test_idx)

model = CNN(32, 32, 3, 10)
trained_model1, _ = train_model(
    model=model,
    dataset=train_dataset_client[0],
    device=device,
    epochs=10)
print('mode 1: ', eval_model(trained_model1, test_dataset, device))
trained_model2, _ = train_model(
    model=model,
    dataset=train_dataset_client[1],
    device=device,
    epochs=10)
print('mode 2: ', eval_model(trained_model2, test_dataset, device))

trained_model3, _ = train_model(
    model=model,
    dataset=train_dataset_client[2],
    device=device,
    epochs=10)
print('mode 3 before distill: ', eval_model(trained_model3, test_dataset, device))

trained_model4, _ = train_model(
    model=trained_model3,
    dataset=public_dataset,
    device=device,
    epochs=10)
print('mode 3 after distill: ', eval_model(trained_model4, test_dataset, device))

trained_model5, _ = train_model_disti(
    model=trained_model3,
    neighbor_server_model=[trained_model1, trained_model2],
    weight=torch.tensor([0.5, 0.5]),
    dataset=public_dataset,
    alpha=0.5,
    device=device,
    epochs=10)
print('mode 3 train only in public dataset: ', eval_model(trained_model5, test_dataset, device))

cuda
Files already downloaded and verified
mode 1:  tensor(0.5055)
mode 2:  tensor(0.5276)
mode 3 before distill:  tensor(0.5098)
mode 3 after distill:  tensor(0.4948)
mode 3 train only in public dataset:  tensor(0.5104)


In [7]:
import torch
import sys

sys.path.append('..')

from utils.data_util import *
from utils.train_util import *
from utils.model_util import *
from utils.lib_util import *

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
print(device)

save_path = '../data/dealed-data/'
train_dataset_o, test_dataset_o = get_dataset(dataset='cifar100')
print(type(train_dataset_o.targets), len(train_dataset_o.targets))

cuda
Files already downloaded and verified
<class 'list'> 50000


In [5]:
import torch
import sys

sys.path.append('..')

from utils.data_util import *
from utils.train_util import *
from utils.model_util import *
from utils.lib_util import *

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
print(device)

save_path = '../data/dealed-data/'
train_dataset_o, test_dataset_o = get_dataset()
print(type(train_dataset_o.targets))

cuda
<class 'torch.Tensor'>


In [8]:
import torch

a = torch.rand(3, 4)
b = a.to('cuda')

print(b)
print(b[0])

tensor([[0.3208, 0.4769, 0.0082, 0.7106],
        [0.9084, 0.5454, 0.7445, 0.4033],
        [0.0428, 0.1435, 0.5771, 0.0324]], device='cuda:0')
tensor([0.3208, 0.4769, 0.0082, 0.7106], device='cuda:0')


In [10]:
import random, copy

a = [i for i in range(10)]
b = [i+100 for i in range(10)]
c = {'a': a, 'b': b}

d = copy.deepcopy(c['a'])
random.shuffle(d)

print(a)
print(b)
print(c)
print(d)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[100, 101, 102, 103, 104, 105, 106, 107, 108, 109]
{'a': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'b': [100, 101, 102, 103, 104, 105, 106, 107, 108, 109]}
[2, 6, 4, 1, 0, 8, 7, 9, 3, 5]


In [12]:
a = [i for i in range(10)]
b = a[1:3]
c = a[1:]
print(a, b, c)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9] [1, 2] [1, 2, 3, 4, 5, 6, 7, 8, 9]


In [11]:
import numpy

a = [i for i in range(10)]
b = numpy.random.choice(a, 3, replace=True).tolist()
print(b, type(b))

for i, c in enumerate(b):
    print(i)
    print(c, type(c))

[8, 8, 5] <class 'list'>
0
8 <class 'int'>
1
8 <class 'int'>
2
5 <class 'int'>
